In [1]:
import os
import rasterio as rio
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString

def convert(tiff_folder, annot_folder, X_folder, Y_folder, prefix):
    os.makedirs(X_folder, exist_ok=True)
    os.makedirs(Y_folder, exist_ok=True)
    
    tiffs = os.listdir(tiff_folder)
    annots = os.listdir(annot_folder)
    for tiff_file, annot_file in zip(tiffs, annots):
        if tiff_file.endswith(".tif") or tiff_file.endswith("x.tiff"):
            # Open the TIFF file
            tiff_path = os.path.join(tiff_folder, tiff_file)
            with rio.open(tiff_path) as src:
                image = src.read().transpose((1, 2, 0))
            image = Image.fromarray(image).resize((1024, 1024))

            # Create the output filename
            base_name = os.path.splitext(tiff_file)[0]
            base_name = base_name.split('/')[-1]
            base_name = base_name.split('_')[1:]
            base_name = '_'.join(base_name)
            output_filename = f"{prefix}_{base_name}.png"
            output_path = os.path.join(X_folder, output_filename)

            # Save the image as PNG
            image.save(output_path)
            
            resolution = 0
            scale = 0
            initial_x = 0
            initial_y = 0
            with tiff.TiffFile(tiff_path) as tif:
                resolution = tif.pages[0].tags[256].value
                scale = tif.pages[0].tags[33550].value[0]
                initial_x = tif.pages[0].tags[33922].value[3]
                initial_y = tif.pages[0].tags[33922].value[4]
        
        if annot_file.endswith(".geojson"):
            try:
                # Read the GeoJSON file
                geojson_path = os.path.join(annot_folder, annot_file)
                lab = gpd.read_file(geojson_path)
                lab.set_crs(epsg=4326, inplace=True)

                # Filter buildings and roads
                buildings = lab[lab['building'] == 'yes']
                primary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "primary")]
                secondary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "secondary")]
                tertiary_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "tertiary")]
                residential_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] == "residential")]
                other_roads = lab[(lab['highway'].notna()) & (lab['building'].isna()) & (lab["highway"] != "primary") & (lab["highway"] != "secondary") & (lab["highway"] != "tertiary") & (lab["highway"] != "residential")]

                # Create a plot
                fig, ax = plt.subplots(figsize=(13.33, 13.33))
                edge_x = [initial_x, initial_x + resolution*scale, initial_x + resolution*scale, initial_x]
                edge_y = [initial_y - resolution*scale, initial_y - resolution*scale, initial_y, initial_y]
                # print(edge_x, edge_y)
                # ax.set_ylim(initial_y, initial_y + resolution*scale)
                # ax.set_xlim(initial_x, initial_x + resolution*scale)

                ax.cla()
                fig.set_facecolor((0, 1, 0, 1))
                # ax = lab[lab['building']=='yes'].plot(color = 'b',label = 'buildings',figsize = (12,12))
                # lab[lab['building']!='yes'].geometry.buffer(0.00001).plot(color = 'r',ax = ax)
                # Plot buildings in 
                
                if not buildings.empty:
                    buildings.plot(ax=ax, color= (0, 0, 1, 1), label='buildings')

                # Plot roads in black
                if not primary_roads.empty:
                    primary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=15, label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=10, label='roads')
                    
                if not secondary_roads.empty:
                    secondary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 8, label='roads')
                    
                if not tertiary_roads.empty:
                    tertiary_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 8, label='roads')
                    
                if not residential_roads.empty:
                    residential_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth= 6, label='roads')
                    
                if not other_roads.empty:
                    other_roads.plot(ax=ax, color=(1, 0, 0, 1), linewidth=3, label='road')
                ax.scatter(edge_x, edge_y, color = (0, 1, 0, 1), marker = '.')
                
                ax.set_axis_off()
                ax.set_aspect('equal')
                # print(fig.get_figwidth())

                # Create the output filename
                base_name = os.path.splitext(annot_file)[0]
                output_filename = f"{prefix}_{base_name}.png"
                output_path = os.path.join(Y_folder, output_filename)
                # print(ax.get_xscale(), ax.get_aspect(), ax.get_xlim(), ax.get_ylim())
                # Save the image
                # plt.gca().set_axis_off()

                plt.margins(0,0)
                plt.savefig(output_path, pad_inches = 0, bbox_inches='tight', dpi = 100)
                plt.plot()
                plt.close()

            except Exception as e:
                print(f"Error processing {annot_file}: {e}")
    
    print("Conversion completed.")
    
    

In [2]:
tiff_folder = 'train/Germany_Training_Public/PRE-event'
X_folder = 'X_images'
annot_folder = 'train/Germany_Training_Public/annotations'
Y_folder = 'Y_train'
prefix = 'Germany'

convert(tiff_folder, annot_folder, X_folder, Y_folder, prefix)

Conversion completed.
